# `tensorflow.keras` convnet from scratch

This notebook demonstrates the use of a simple convolutional network. On a 2-class classification task--that is, an ouput layer with two nodes activated via `softmax`. Accordingly, loss is calculated using the `categorical cross-entropy`.

References:

* The github repository from the BAGLS team contained in this [link](https://github.com/anki-xyz/bagls/blob/master/Utils/DataGenerator.py#L109)


In [1]:
# dev convenience
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")
import PATHS

import os
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "0" # use only this GPU device
# os.environ["WANDB_SILENT"] = "True"
os.environ["WANDB_NOTEBOOK_NAME"] = "03-simple-convnet.ipynb"

PROJECT_NAME = 'bagls-sh-test'
RUN_NAME = 'convnet_from_scratch'
METRICS_TABLE_NAME = 'metrics_table'
GRADCAM_LAYER_NAME = "conv2d_3"

In [3]:
import wandb
print("W&B: ", wandb.__version__)
wandb.login()

# # manage logs
import logging

# logger = logging.getLogger("wandb")
# logger.setLevel(logging.ERROR)

logging.getLogger('tensorflow').disabled = True

W&B:  0.13.5


wandb: Currently logged in as: miked. Use `wandb login --relogin` to force relogin


In [4]:
# tf loader
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 975858170141486177
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10925703168
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1320403930953056154
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1"
]


2022-11-30 11:34:20.120767: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 11:34:21.033966: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1


In [5]:
import config
configs = config.nb_configs

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.imagenet_utils import preprocess_input

# initialize data generator
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=configs["validation_split"],
    rescale=configs["rescale"],
    width_shift_range=configs["width_shift_range"],
    height_shift_range=configs["height_shift_range"],
    shear_range=configs["shear_range"],
    zoom_range=configs["zoom_range"],
    fill_mode=configs["fill_mode"],
    horizontal_flip=configs["horizontal_flip"],
    rotation_range=configs["rotation_range"],
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, 
    rescale=configs["rescale"],
)

In [7]:
train_dir = configs["train_dir"]
test_dir = configs["test_dir"]

batch_size = configs["batch_size"]
class_names = configs["class_names"]
interpol = configs["interpol"]
cmap = configs["cmap"]
label_mode = configs["label_mode"]
labels = configs["labels"]
image_size = configs["image_size"]


train_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="training",
)

val_dataset = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    subset="validation",
)

test_dataset = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=image_size,
    color_mode=cmap,
    classes=class_names,
    class_mode=label_mode,
    batch_size=batch_size,
    interpolation=interpol,
    shuffle=False, # do not shuffle for later evaluation, alphanum sort
)

configs.update({"val_steps": val_dataset.samples // configs["batch_size"]})

Found 526 images belonging to 2 classes.
Found 26 images belonging to 2 classes.
Found 33 images belonging to 2 classes.


In [8]:
from tensorflow import keras
from tensorflow.keras import layers

def define_model():
    inputs = keras.Input(shape=(*image_size, 3))
    x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(units=512, activation='relu')(x)
    x = keras.layers.Dropout(configs['dropout_rate'])(x)
    outputs = layers.Dense(units=2, activation='softmax')(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [9]:
tf.keras.backend.clear_session()
model = define_model()

2022-11-30 11:34:21.329196: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1


In [10]:
from tensorflow.keras import metrics
thresh = configs["thresh"]
metrics_dict = {
    "ACC":  metrics.CategoricalAccuracy(name="ACC"),
    "AUC-ROC": metrics.AUC(name='ROC', curve='ROC'),
    "AUC-PR": metrics.AUC(name='PR', curve='PR'),
    "TP": metrics.TruePositives(name="TP", thresholds=thresh),
    "TN": metrics.TrueNegatives(name="TN", thresholds=thresh),
    "FP": metrics.FalsePositives(name="FP", thresholds=thresh),
    "FN": metrics.FalseNegatives(name="FN", thresholds=thresh),
}

In [11]:
from tensorflow.keras import optimizers
from tensorflow.keras import losses

# opt = optimizers.Adam(learning_rate=1e-06)
opt = optimizers.Adam()
met = list(metrics_dict.values())

model.compile(
    loss=losses.CategoricalCrossentropy(),
    optimizer=opt,
    metrics=met,
)

In [12]:
# verify arch
# base predictions with untrained classif head
base_preds = model.predict(test_dataset)
base_preds

2022-11-30 11:34:29.687638: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-11-30 11:34:31.008964: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8400


array([[0.49639297, 0.50360703],
       [0.5001336 , 0.49986643],
       [0.5003542 , 0.49964586],
       [0.49822372, 0.5017762 ],
       [0.495909  , 0.504091  ],
       [0.49226582, 0.5077342 ],
       [0.48783743, 0.51216257],
       [0.48677975, 0.5132202 ],
       [0.48987347, 0.51012653],
       [0.48297465, 0.51702535],
       [0.4887182 , 0.5112818 ],
       [0.48798332, 0.5120167 ],
       [0.487761  , 0.51223904],
       [0.48332182, 0.51667815],
       [0.48200056, 0.5179995 ],
       [0.48334572, 0.51665425],
       [0.48804355, 0.51195645],
       [0.48216254, 0.51783746],
       [0.4838706 , 0.51612943],
       [0.48243693, 0.5175631 ],
       [0.4853602 , 0.5146398 ],
       [0.48073095, 0.51926905],
       [0.48196772, 0.51803225],
       [0.47921944, 0.5207806 ],
       [0.49530894, 0.50469106],
       [0.47898576, 0.5210142 ],
       [0.48430455, 0.51569545],
       [0.48169604, 0.518304  ],
       [0.4905136 , 0.5094864 ],
       [0.4899489 , 0.51005113],
       [0.

In [13]:
from interpretation import ValLog, GRADCamLogger
# initialize run
run = wandb.init(
    project=PROJECT_NAME, 
    name=RUN_NAME,
    config=configs, 
    job_type='train',
)

wandb_callback = wandb.keras.WandbCallback(
    monitor="val_ROC",
    mode="max",
    save_model=True,
    save_graph=True,
    compute_flops=True,
)

callbacks = [
    wandb_callback,
    ValLog(generator=val_dataset, num_log_batches=1),
    GRADCamLogger(generator=test_dataset, layer_name=GRADCAM_LAYER_NAME, num_log_batches=1)
    
]

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [14]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=configs["epochs"], 
    shuffle=True,
    callbacks=callbacks,
)
run.finish()

2022-11-30 11:35:00.152124: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-11-30 11:35:00.152343: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-11-30 11:35:00.154772: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10419 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1
2022-11-30 11:35:00.158365: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.007ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.



Epoch 1/5
5/5 [==============================] - 12s 2s/step - loss: 0.9456 - ACC: 0.5190 - ROC: 0.5743 - PR: 0.5780 - TP: 273.0000 - TN: 273.0000 - FP: 253.0000 - FN: 253.0000 - val_loss: 0.7114 - val_ACC: 0.3846 - val_ROC: 0.2766 - val_PR: 0.3772 - val_TP: 10.0000 - val_TN: 10.0000 - val_FP: 16.0000 - val_FN: 16.0000


2022-11-30 11:35:13.717913: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
wandb: Adding directory to artifact (/home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221130_113440-djzb7dq0/files/model-best)... Done. 0.7s


Epoch 2/5
5/5 [==============================] - 10s 2s/step - loss: 0.6974 - ACC: 0.5000 - ROC: 0.4547 - PR: 0.4590 - TP: 263.0000 - TN: 263.0000 - FP: 263.0000 - FN: 263.0000 - val_loss: 0.6871 - val_ACC: 0.6154 - val_ROC: 0.5976 - val_PR: 0.5626 - val_TP: 16.0000 - val_TN: 16.0000 - val_FP: 10.0000 - val_FN: 10.0000


wandb: Adding directory to artifact (/home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221130_113440-djzb7dq0/files/model-best)... Done. 0.7s


Epoch 3/5
5/5 [==============================] - 9s 2s/step - loss: 0.6728 - ACC: 0.6065 - ROC: 0.6330 - PR: 0.6139 - TP: 319.0000 - TN: 319.0000 - FP: 207.0000 - FN: 207.0000 - val_loss: 0.6603 - val_ACC: 0.6154 - val_ROC: 0.6982 - val_PR: 0.6429 - val_TP: 16.0000 - val_TN: 16.0000 - val_FP: 10.0000 - val_FN: 10.0000


wandb: Adding directory to artifact (/home/mdorosan/2022/bagls-sh-project/notebooks/wandb/run-20221130_113440-djzb7dq0/files/model-best)... Done. 0.9s


Epoch 4/5
5/5 [==============================] - 9s 2s/step - loss: 0.6719 - ACC: 0.6065 - ROC: 0.6381 - PR: 0.6190 - TP: 319.0000 - TN: 319.0000 - FP: 207.0000 - FN: 207.0000 - val_loss: 0.6532 - val_ACC: 0.6154 - val_ROC: 0.6775 - val_PR: 0.6252 - val_TP: 16.0000 - val_TN: 16.0000 - val_FP: 10.0000 - val_FN: 10.0000
Epoch 5/5
5/5 [==============================] - 9s 2s/step - loss: 0.6588 - ACC: 0.6084 - ROC: 0.6458 - PR: 0.6272 - TP: 320.0000 - TN: 320.0000 - FP: 206.0000 - FN: 206.0000 - val_loss: 0.6343 - val_ACC: 0.6154 - val_ROC: 0.6953 - val_PR: 0.6774 - val_TP: 16.0000 - val_TN: 16.0000 - val_FP: 10.0000 - val_FN: 10.0000


ACC,▂▁███
FN,▇█▁▁▁
FP,▇█▁▁▁
PR,▆▁▇██
ROC,▅▁███
TN,▂▁███
TP,▂▁███
epoch,▁▃▅▆█
loss,█▂▁▁▁
val_ACC,▁████
val_FN,█▁▁▁▁


In [ ]:
run = wandb.init(project=PROJECT_NAME, job_type="inference", name=RUN_NAME)
model_at = run.use_artifact("model-" + RUN_NAME + ":latest")
model_dir = model_at.download()
print("model: ", model_dir)
best_model = keras.models.load_model(model_dir)

In [ ]:
# test metrics
import utils

metrics_results = best_model.evaluate(test_dataset)
metrics_results = dict(zip(["loss"] + list(metrics_dict.keys()), 
                           metrics_results))
tp, fp, tn, fn = (metrics_results["TP"], metrics_results["FP"], 
                  metrics_results["TN"], metrics_results["FN"])

add_metrics = {
    "SENSITIVITY": utils.get_sensitivity(tp, fp, tn, fn),
    "SPECIFICTY": utils.get_specificity(tp, fp, tn, fn),
    "PPV": utils.get_ppv(tp, fp, tn, fn),
    "NPV": utils.get_npv(tp, fp, tn, fn),
    "F1" : utils.get_fbeta(tp, fp, tn, fn, beta=1),
}
metrics_results.update(add_metrics)

print(f"Metrics: \n", metrics_results)

columns = list(metrics_results.keys())
metrics_table = wandb.Table(columns=columns)
metrics_table.add_data(*metrics_results.values())
wandb.run.log({METRICS_TABLE_NAME : metrics_table})

# add logging of confusion matrix image from matplotlib
           
# get preds
trained_preds = best_model.predict(test_dataset)
run.finish()

## End